In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import numpy as np

In [75]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
trigrams = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2]) for i in range(len(test_sentence) - 2)]

In [76]:
vocab = set(test_sentence)
word_2_idx_dic = {word: no for no, word in enumerate(vocab)}

In [82]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, trigrams):
        contexts = []
        next_words = []
        for context, next_word in trigrams:
            contexts.append([word_2_idx_dic[word] for word in context])
            next_words.append(word_2_idx_dic[next_word])

        contexts = np.array(contexts, dtype=np.int64)
        next_words = np.array(next_words, dtype=np.int64)

        self.contexts = contexts
        self.next_words = next_words
    
    def __len__(self):
        return len(self.next_words)
    
    def __getitem__(self, i):
        return self.contexts[i], self.next_words[i]
    
train_data = MyDataSet(trigrams)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

In [83]:
class NGramLM(nn.Module):
    def __init__(self, vocab_size, emb_size, context_len, hidden_size):
        super(NGramLM, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.fc1 = nn.Linear(context_len * emb_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, context_batch):
        x = self.emb(context_batch).view(context_batch.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = NGramLM(len(vocab), 10, 2, 128)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [85]:
losses = []
for epoch_no in range(30):
    loss_sum = 0.
    for data in train_loader:
        optimizer.zero_grad()
        context_batch, next_word_batch = data
        output = net.forward(context_batch)
        loss = criterion(output, next_word_batch)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
    losses.append(loss_sum)
print(losses)

[299.135971814394, 272.8966073691845, 246.3786700218916, 220.39922662079334, 195.22155153751373, 171.23473075032234, 149.0150276273489, 128.6622850522399, 110.86262924969196, 95.62582163512707, 82.4597737044096, 71.31865440309048, 61.85314521193504, 54.14468117803335, 47.848887398839, 42.51327008754015, 38.23937687277794, 34.599759079515934, 31.468769513070583, 29.00860485434532, 26.800600700080395, 24.97014182806015, 23.2825563326478, 22.016695730388165, 20.677562549710274, 19.627597589045763, 18.854329355061054, 17.769334971904755, 16.95172924362123, 16.379757076501846]
